# tiktokenを使ったトークンのカウント方法について

[`tiktoken`](https://github.com/openai/tiktoken/blob/main/README.md)はOpenAIによる高速なオープンソース・トークナイザーです。

テキスト列 (例: `"tiktoken is great!"`) とエンコーディング (例: `"cl100k_base"`) が与えられた場合、トークナイザーはテキスト列をトークンのリストに分割できます (例: `["t", "iik", "token", " is", " great", "!"]`).

テキスト文字列のトークンへの分割は、GPTモデルがテキストをトークンの形で参照するために有用です。テキスト文字列の中にいくつのトークンがあるかを知ることで (a) 文字列が長すぎてテキストモデルが処理できないか、(b) OpenAIのAPI呼び出しがいくらかかるか（利用料金はトークン単位なので）がわかります。


## エンコーディング

エンコーディングは、テキストがトークンに変換される方法を指定します。異なるモデルは異なるエンコーディングを使用します。

`tiktoken` は、OpenAI モデルで使用される3つのエンコーディングをサポートしています。

| エンコーディング名                | OpenAI モデル                                         |
|--------------------------|----------------------------------------------------|
| `cl100k_base`            | `gpt-4`, `gpt-3.5-turbo`, `text-embedding-ada-002` |
| `p50k_base`              | Codex モデル、`text-davinci-002`、`text-davinci-003`    |
| `r50k_base` (または `gpt2`) | `davinci` などの GPT-3 モデル                            |

以下のように `tiktoken.encoding_for_model()` を使用して、モデルのエンコーディングを取得できます。

```python
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
```

`p50k_base` は `r50k_base` と大幅に重複しており、非コードのアプリケーションでは通常、同じトークンが得られます。

## 言語別のトークナイザーライブラリ

`cl100k_base` と `p50k_base` のエンコーディングについては、以下のライブラリがあります。

- Python: [tiktoken](https://github.com/openai/tiktoken/blob/main/README.md)
- .NET / C#: [SharpToken](https://github.com/dmitry-brazhenko/SharpToken), [TiktokenSharp](https://github.com/aiqinxuancai/TiktokenSharp)
- Java: [jtokkit](https://github.com/knuddelsgmbh/jtokkit)

`r50k_base` (`gpt2`) のエンコーディングについては、多くの言語でトークナイザーが利用可能です。

- Python: [tiktoken](https://github.com/openai/tiktoken/blob/main/README.md) (または [GPT2TokenizerFast](https://huggingface.co/docs/transformers/model_doc/gpt2#transformers.GPT2TokenizerFast))
- JavaScript: [gpt-3-encoder](https://www.npmjs.com/package/gpt-3-encoder)
- .NET / C#: [GPT Tokenizer](https://github.com/dluc/openai-tools)
- Java: [gpt2-tokenizer-java](https://github.com/hyunwoongko/gpt2-tokenizer-java)
- PHP: [GPT-3-Encoder-PHP](https://github.com/CodeRevolutionPlugins/GPT-3-Encoder-PHP)

（OpenAI は、サードパーティのライブラリについて、いかなる保証も行いません。）

## 一般的な文字列のトークナイズ方法

英語では、トークンの長さは通常、1文字から1単語までの範囲です（例： `"t"` または `" great"`）。ただし、一部の言語では、トークンが1文字未満または1単語より長くなる場合があります。スペースは通常、単語の先頭とグループ化されます（例： `" is"` ではなく `"is "` または `" "`+`"is"`）。文字列がどのようにトークナイズされるかを簡単に確認できるのは、[OpenAI Tokenizer](https://beta.openai.com/tokenizer) です。

## 0. `tiktoken`のインストール

必要に応じて、`pip`を使用して`tiktoken`をインストールしてください。

In [ ]:
%pip install --upgrade tiktoken
%pip install --upgrade openai

## 1. `tiktoken`をインポートする。

In [ ]:
import tiktoken

## 2. エンコーディングをロードする

`tiktoken.get_encoding()` を使用して、名前でエンコーディングをロードします。

初回実行時には、ダウンロードにインターネット接続が必要です。以降の実行では、インターネット接続は必要ありません。

In [ ]:
encoding = tiktoken.get_encoding("cl100k_base")

与えられたモデル名に対して正しいエンコーディングを自動的に読み込むには、 `tiktoken.encoding_for_model()` を使用します。

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

## 3. encoding.encode()`でテキストをトークンに変換する。



`.encode()` メソッドは、文字列をトークン整数のリストに変換します。

In [ ]:
encoding.encode("tiktokenは素晴らしい!")

`.encode()` が返すリストの長さを数えて、トークンを数える。

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """テキスト文字列中のトークンの数を返します。"""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
num_tokens_from_string("tiktoken is great!", "cl100k_base")

## 4. `encoding.decode()` でトークンをテキストに変換する。

`.decode()` は、トークン整数のリストを文字列に変換します。

In [ ]:
encoding.decode([83, 1609, 5963, 374, 2294, 0])

Warning: `.decode()` は単一のトークンに適用できますが、utf-8の領域にないトークンに対しては非可逆的であることに注意してください。

単一トークンの場合、`.decode_single_token_bytes()`は単一の整数のトークンを、それが表すバイト列に安全に変換します。

In [ ]:
[encoding.decode_single_token_bytes(token) for token in [83, 1609, 5963, 374, 2294, 0]]

(文字列の前にある`b`はバイト文字列であることを示す)

## 5. エンコーディングの比較

エンコーディングの種類によって、単語の分割、スペースのグループ化、非英語文字の扱い方などが異なります。上記の方法を用いて、少数のサンプル文字列で異なるエンコーディングを比較することができます。

In [ ]:
def compare_encodings(example_string: str) -> None:
    """3つの文字列のエンコーディングを比較した結果を表示します。"""
    # サンプル文字列を表示する
    print(f'\nExample string: "{example_string}"')
    # 各エンコーディングについて、トークンの数、トークンの整数値、およびトークンのバイトを出力します。
    for encoding_name in ["gpt2", "p50k_base", "cl100k_base"]:
        encoding = tiktoken.get_encoding(encoding_name)
        token_integers = encoding.encode(example_string)
        num_tokens = len(token_integers)
        token_bytes = [encoding.decode_single_token_bytes(token) for token in token_integers]
        print()
        print(f"{encoding_name}: {num_tokens} tokens")
        print(f"token integers: {token_integers}")
        print(f"token bytes: {token_bytes}")
        

In [ ]:
compare_encodings("antidisestablishmentarianism")

In [ ]:
compare_encodings("2 + 2 = 4")

In [ ]:
compare_encodings("お誕生日おめでとう")

## 6. チャットAPIの呼び出しでトークンを数える

ChatGPTモデル（`gpt-3.5-turbo`や`gpt-4`など）は、古い補完モデルと同じようにトークンを使用しますが、メッセージベースのフォーマットのため、会話で使用されるトークンの数を数えることがより困難になっています。

以下は、`gpt-3.5-turbo-0301`または`gpt-4-0314`に渡されたメッセージのトークンをカウントするためのサンプル関数です。

メッセージからトークンをカウントする正確な方法は、モデルによって変化する可能性があることに注意してください。以下の関数によるカウントは推定値であり、時代を超えて保証されるものではないとお考えください。

In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """メッセージのリストで使用されるトークンの数を返します。"""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("警告：モデルが見つかりません。cl100k_baseエンコーディングを使用しています。")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo":
        print("警告：gpt-3.5-turboは時間の経過とともに変更される可能性があります。gpt-3.5-turbo-0301を仮定してトークン数を返します。")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301")
    elif model == "gpt-4":
        print("警告：gpt-4は時間の経過とともに変更される可能性があります。gpt-4-0314を仮定してトークン数を返します。")
        return num_tokens_from_messages(messages, model="gpt-4-0314")
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # 各メッセージは<|start|>{role/name}\n{content}<|end|>\nに従います。
        tokens_per_name = -1  # 名前がある場合、roleは省略されます。
    elif model == "gpt-4-0314":
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(f"""num_tokens_from_messages()は、モデル{model}に対して実装されていません。メッセージがトークンに変換される方法については、https://github.com/openai/openai-python/blob/main/chatml.mdを参照してください。""")
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # 各返信は<|start|>assistant<|message|>でプライムされます。
    return num_tokens

In [ ]:
# 上記の関数がOpenAI APIのレスポンスと一致することを確認しましょう。
import openai

example_messages = [
    {
        "role": "system",
        "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "New synergies will help drive top-line growth.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Things working well together will increase revenue.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Let's talk later when we're less busy about how to do better.",
    },
    {
        "role": "user",
        "content": "This late pivot means we don't have time to boil the ocean for the client deliverable.",
    },
]

In [ ]:
for model in ["gpt-3.5-turbo-0301", "gpt-4-0314"]:
    print(model)
    # 上記で定義された関数によるサンプルトークン数
    print(f"{num_tokens_from_messages(example_messages, model)} prompt tokens counted by num_tokens_from_messages().")
    # OpenAI APIからのトークン数の例
    response = openai.ChatCompletion.create(
        model=model,
        messages=example_messages,
        temperature=0,
        max_tokens=1  # ここでは入力トークンのみをカウントしているので、出力に無駄なトークンを使わないようにしましょう
    )
    print(f'{response["usage"]["prompt_tokens"]} prompt tokens counted by the OpenAI API.')
    print()
